<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
RandomizedSearch - Random Forest - Diamonds
</b></font> </br></p>

---


In [ ]:
#@title 🔧 Colab-Umgebung { display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from ml_lib.utilities import get_ipinfo
import sys
print()
print(f"Python Version: {sys.version}")
print()
get_ipinfo()

# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import read_csv, DataFrame

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

import plotly.express as px
import plotly.subplots as sp

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
Anwendungsfall
</font></p>

---

Dieser klassische Datensatz enthält die Preise und andere Attribute von fast 54.000 Diamanten.



[DataSet](https://www.openml.org/search?type=data&status=active&id=42225)

[Info](https://www.kaggle.com/datasets/shivam2503/diamonds)


In [ ]:
df = read_csv(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/diamonds.csv",
    usecols=[
        "carat",
        "cut",
        "color",
        "clarity",
        "depth",
        "table",
        "price",
    ],
)

In [ ]:
data = df.copy()
target = data.pop("price")

<p><font color='black' size="5">
EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.groupby("cut").count()

In [ ]:
data.groupby("color").count()

<p><font color='black' size="5">
EDA (Exploratory Data Analysis) mit Plotly
</font></p>

In [ ]:
title_ = "Depth"
b1 = px.box(data["depth"], title=title_, width=600, height=600)

title_ = "Carat"
b2 = px.box(data["carat"], title=title_, width=600, height=600)

title_ = "Table"
b3 = px.box(data["table"], title=title_, width=600, height=600)

fig = sp.make_subplots(rows=1, cols=3, subplot_titles=("Depth", "Carat", "Table"))

for trace in b1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in b2.data:
    fig.add_trace(trace, row=1, col=2)

for trace in b3.data:
    fig.add_trace(trace, row=1, col=3)

# Layout anpassen
fig.update_layout(width=1000, height=500, title_text="Box-Plots")

# Plot anzeigen
fig.show()

# 2 |  Prepare

---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Duplikate ermitteln/löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>
✅ Train-Test-Split durchführen</br>

<p><font color='black' size="5">
Datentyp ermitteln
</font></p>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<p><font color='black' size="5">
Kodierung
</font></p>

In [ ]:
cat_seq = [
    ["Fair", "Good", "Very Good", "Premium", "Ideal"],
    ["J", "I", "H", "G", "F", "E", "D"],
    ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"],
]
coder = OrdinalEncoder(categories=cat_seq)
data[cat_col] = coder.fit_transform(data[cat_col])

<p><font size="5">
Train-Test-Split
</p>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.30, random_state=42
)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3 | Modeling
---

<p><font size="5">
Modelauswahl & Training
</p>

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

In [ ]:
model = RandomForestRegressor()

<p><font size="5">
Hyperparameter Tuning - RandomizedSearch
</p>

In [ ]:
parameter_grid = {}
parameter_grid["n_estimators"] = range(10, 500, 10)
parameter_grid["max_features"] = range(1, 10, 1)
parameter_grid["max_depth"] = range(10, 30, 10)
parameter_grid["min_samples_leaf"] = range(5, 20, 5)

In [ ]:
tuned_model = RandomizedSearchCV(
    estimator=model,
    param_distributions=parameter_grid,
    n_iter=10,
    cv=2,
    scoring="r2",
    return_train_score=True,
    verbose=1,
    random_state=42,
)

In [ ]:
tuned_model.fit(data_train, target_train)

<p><font color='black' size="5">
Alle Ergebnisse RandomizedSearch
</font></p>

In [ ]:
tuned_model.cv_results_

<p><font color='black' size="5">
Ergebnisse der Hyperparameter
</font></p>

In [ ]:
tuned_model.best_estimator_

In [ ]:
tuned_model.best_params_

In [ ]:
tuned_model.best_score_

In [ ]:
# Erstellen eines DataFrame direkt aus den relevanten Teilen des Dictionaries
results = DataFrame(
    {
        "MTS": tuned_model.cv_results_["mean_test_score"],
        "Parameter": tuned_model.cv_results_["params"],
    }
)
results

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Prognose auf Basis des besten Modells
</font></p>

In [ ]:
target_train_pred = tuned_model.predict(data_train)

In [ ]:
target_test_pred = tuned_model.predict(data_test)

<p><font color='black' size="5">
Bestimmtheitsmass
</font></p>

In [ ]:
r2 = r2_score(target_train, target_train_pred)
print(f"-- Train --- Bestimmtheitsmass: {r2:5.2f}")

In [ ]:
r2 = r2_score(target_test, target_test_pred)
print(f"-- Test --- Bestimmtheitsmass: {r2:5.2f}")

<p><font color='black' size="5">
Mean Absolut Error
</font></p>

In [ ]:
mae = mean_absolute_error(target_test, target_test_pred)
print(f"-- Test -- Mean Absolute Error: {mae:5.2f}")

<p><font color='black' size="5">
Aufbau Analysewürfel
</font></p>

In [ ]:
# Übernahme der Testdaten
cube = data_test.copy()
cube.reset_index(inplace=True)

# Übernahem Target real & predict
cube["real"] = DataFrame(target_test.values, columns=["real"])
cube["predict"] = DataFrame(target_test_pred, columns=["predict"])

<p><font color='black' size="5">
Visualisierung real vs predict
</font></p>

In [ ]:
# Boxplot
title_ = "Boxplot real vs predict"
px.box(cube[["real", "predict"]], title=title_, width=600, height=600)

In [ ]:
# Histogramm
title_ = "Histogramm Prices real vs predict"
fig = px.histogram(cube, x=["real", "predict"], nbins=10, title=title_)
fig.update_layout(barmode="group", bargap=0.2, width=800, height=600)
fig.show()

<p><font color='black' size="5">
Fehlerhafte Vorhersagen
</font></p>

In [ ]:
cube["abs_Abw%"] = abs((cube["real"] - cube["predict"]) / cube["real"] * 100)
%precision 3
cube.head(10).style.format("{:,.1f}")

In [ ]:
cube.describe().T

In [ ]:
# Histogramm
title_ = "Histogramm absolute Abweichung"
fig = px.histogram(cube, x=["abs_Abw%"], nbins=20, text_auto=".2s", title=title_)
fig.update_layout(barmode="group", bargap=0.2, width=800, height=600)
fig.show()

<p><font color='black' size="5">
Feature Importance
</font></p>

In [ ]:
px.bar(
    x=tuned_model.best_estimator_.feature_importances_,
    y=tuned_model.best_estimator_.feature_names_in_,
).update_yaxes(categoryorder="total ascending")

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>